<a href="https://colab.research.google.com/github/wnrud1225/webcam-image-save/blob/main/webcam_image_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date, datetime
from torchvision import models
import torch
import torch.nn as nn
from math import floor
import cv2
import numpy as np
import os
import torchvision.transforms as transforms
webcam = cv2.VideoCapture(0)

print(webcam)
if not webcam.isOpened():
    print("Could not open webcam")
    exit()

captureTime = 0
# Model
def import_model():
    model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.DEFAULT)  
    model.classifier[1] = nn.Linear(1536, 4)
    return model
model = import_model()
print(model)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])
while webcam.isOpened():
    status, frame = webcam.read()
    if status:
        img = frame
        if captureTime + 0.08 > datetime.now().timestamp():
            white = np.zeros_like(frame, dtype=np.uint8)
            white.fill(255)
            img = cv2.addWeighted(frame, 1, white, 0.1, 1)
        cv2.imshow("test", img)
        img = transform(img).unsqueeze(0)
        predicted = model(img)
        prediction = torch.argmax(predicted)
        if prediction ==0:
            out = 'stop'
        if prediction ==1:
            out = 'next'
        if prediction ==2:
            out = 'down'
        if prediction ==3:
            out = 'none'
        print(out)

    if cv2.waitKey(1) == ord('c'):
        save_name = os.path.join('data/test/0', f'capture_{floor(datetime.now().timestamp())}.png' )
        captured = cv2.imwrite(save_name, frame)
        captureTime = datetime.now().timestamp()
        print(f"Saved {save_name}")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()